<a href="https://colab.research.google.com/github/AmineSdk/RecommenderSystem/blob/main/Notebook/CF_Social_Autoencoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [7]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
# ! kaggle datasets download yelp-shopping-rating-review -f review_shopping.csv
# ! kaggle datasets download Yelp-shopping-GMF-filled-mat -f Yelp_shopping_GMF_filled_mat.csv
! kaggle datasets download yelp-restaurants-review-rating -f review_restaurant.csv
#! kaggle datasets download Aamazon-Cellphones-35k-GMF-filled-mat -f Aamazon_Cellphones_35k_GMF_filled_mat.csv
#! kaggle datasets download Yelp-restaurants-GMF-filled-mat -f Yelp_restaurants_GMF_filled_mat.csv

 96% 145M/152M [00:02<00:00, 84.3MB/s]
100% 152M/152M [00:02<00:00, 78.0MB/s]


In [ ]:
!pip install tensorflow-ranking

In [9]:
from sklearn import preprocessing
from scipy.sparse import csr_matrix
import tensorflow as tf 
import numpy as np
import tensorflow_hub as hub
import pandas as pd
import random
from keras import datasets, layers, models
from sklearn.model_selection import train_test_split
from progressbar import progressbar
from sklearn.compose import ColumnTransformer

In [3]:
def amazonPreprocess(df):
  
  df = df[:35000]
  df = df.dropna(how='any',axis=0)
  df.rename(columns = {'reviewerID':'userID', 'productID' : 'itemID'},inplace = True)
  df.drop_duplicates(subset =['itemID', 'userID'] , keep = 'first' , inplace = True)
  print("DATASET PREPROCESSED")

  return df


In [13]:
def yelpPreprocess(df):

  df = df.dropna(how='any',axis=0)
  df.drop(columns=['funny', 'cool','review_id','useful','date'],inplace=True)
  df.rename(columns = {'user_id':'userID', 'business_id':'itemID','stars':'rating'},inplace = True)
  df.drop_duplicates(subset =['itemID', 'userID'] , keep = 'first' , inplace = True)
  print("DATASET PREPROCESSED")

  return df


In [5]:
def DatasetToUserItemDataFrame(dataframe,userID,itemID,rating):
  #Setting new item IDs from string to int 
  itemKeys = [] 
  i = 0
  for item in dataframe[itemID].value_counts(sort=False):
    temp = np.full((item),i)
    itemKeys = np.append(itemKeys,temp)
    i += 1

  #Setting new user IDs from string to int
  userKeysDic = {}
  userKeys = np.zeros((dataframe[userID].size))
  i = 0
  for user in dataframe[userID].unique():
    userKeysDic[user] = i
    i += 1
  i = 0
  for user in dataframe[userID]:
    userKeys[i] = userKeysDic[user]
    i += 1

  #Converting arrays from float to int 
  userKeys = userKeys.astype(int)
  itemKeys = itemKeys.astype(int)
  print(userKeys.size)
  print(itemKeys.size)
  
  user_item = csr_matrix((dataframe[rating].values.astype(int),(userKeys,itemKeys))) #Creating sparse matrix
  user_item_matrix = user_item.toarray() #Converting sparse matrix into array
  df_user_item = pd.DataFrame(user_item_matrix,index = dataframe[userID].unique()  ,columns = dataframe[itemID].unique() ) 

  return df_user_item

def getUsersRatings(df,usersList): #returns df_rating containing only users that have friends 
  temp = df.copy()
  for user in list(df['userID']):
    if user not in usersList:
      temp = temp.drop(temp.loc[temp['userID'] == user].index)
  return temp

def genIndexColumn(df,min_index,max_index,min_column,max_column): #generates random int index and columns for a given df
  New_User_IDs = random.sample(range(min_index,max_index),df.index.size)
  New_Item_IDs = random.sample(range(min_column,max_column),df.columns.size)

  df_new = pd.DataFrame(df.to_numpy(),index = New_User_IDs,columns = New_Item_IDs)
  return df_new

def generateIDs(df,index,columns,min_index,max_index,min_column,max_column):
  users = df[index].unique()
  items = df[columns].unique()
  df_train = df.copy()

  New_User_IDs = random.sample(range(min_index,max_index),df[index].nunique())
  New_Item_IDs = random.sample(range(min_column,max_column),df[columns].nunique())
  i = 0
  for d in progressbar(users) :
    df_train[index].replace({d : New_User_IDs[i]}, inplace=True)
    i+=1

  i = 0
  for d in progressbar(items) :
    df_train[columns].replace({d : New_Item_IDs[i]}, inplace=True)
    i+=1
  return df_train

def processGmfData(df,index,column,min_user_id,max_user_id,min_item_id,max_item_id):
  
  
  df.rename(columns = {'text': 'reviewText', 'stars': 'rating', 'business_id': 'itemID', 'user_id': 'userID'}, inplace = True)
  #df.rename(columns = {'reviewerID':'userID', 'productID' : 'itemID'},inplace = True)
  df = generateIDs(df,index,column,min_user_id,max_user_id,min_item_id,max_item_id)
  print("== PREPROCESSING DATA ...")
  df = df.dropna(how='any',axis=0)
  df.drop_duplicates(subset =['itemID', 'userID'] , keep = 'first' , inplace = True)

  df['one'] = df['rating'].apply(lambda x: 1 if x==1 else 0)
  df['two'] = df['rating'].apply(lambda x: 1 if x==2 else 0)
  df['three'] = df['rating'].apply(lambda x: 1 if x==3 else 0)
  df['four'] = df['rating'].apply(lambda x: 1 if x==4 else 0)
  df['five'] = df['rating'].apply(lambda x: 1 if x==5 else 0)
  #df['six'] = df['rating'].apply(lambda x: 1 if x==6 else 0)
  print("== DATA PREPROCESSED ==")

  return df

In [27]:
#GMF
def getGMFmodel(num_users,num_items,SIZE_):

  input_userID = layers.Input(shape=[1], name='user_ID')
  input_itemID = layers.Input(shape=[1], name='item_ID')

  user_emb_GMF = layers.Embedding(num_users, SIZE_, name='user_emb_GMF')(input_userID)
  item_emb_GMF = layers.Embedding(num_items, SIZE_, name='item_emb_GMF')(input_itemID)

  u_GMF = layers.Flatten()(user_emb_GMF)
  i_GMF = layers.Flatten()(item_emb_GMF)

  dot_layer = layers.Multiply()([u_GMF, i_GMF])

  out_layer = layers.Dense(1, activation='sigmoid', name='output')(dot_layer)

  GMF = tf.keras.Model([input_userID, input_itemID], out_layer)
  
  return GMF


def user_item_ID_lists(userIDs,itemIDs):
  i = 0
  item_s = pd.Series()
  user_s = pd.Series()

  if userIDs.size > itemIDs.size:
    for item in progressbar(itemIDs):
      temp = pd.Series(userIDs)
      user_s = user_s.append(temp)
      temp = []
      temp = [item for user in userIDs]
      temp = pd.Series(temp)
      item_s = item_s.append(temp)
  else:
    for user in progressbar(userIDs):
      temp = pd.Series(itemIDs)
      item_s = item_s.append(temp)
      temp = []
      temp = [user for item in itemIDs]
      temp = pd.Series(temp)
      user_s = user_s.append(temp)

  return user_s,item_s


def Fill_Cf_Matrix(model,userList,itemList,userIDs,itemIDs):
  rowLen = userIDs.size if userIDs.size > itemIDs.size else itemIDs.size

  prediction = model.predict([userList,itemList],verbose = 1)
  i = 0
  row = []
  matrix = []
  print("pred done")
  
  for i in progressbar(range(userList.shape[0])):
    #result = np.where(prediction[i] == np.amax(prediction[i]))[0][0] + 1
    result = prediction[i][0]
    row.append(result)
    if len(row) == rowLen:
      matrix.append(row)
      row = []
       
    i += 1

  matrix_arr = np.array(matrix)
  if(userIDs.size > itemIDs.size):
    matrix_arr = matrix_arr.transpose()
  dataframe = pd.DataFrame(matrix_arr, index = userIDs, columns = itemIDs)
  
  return dataframe

def loadDataset(fileID):
  dataset = pd.read_csv(DATA[fileID])
  print("== FILE LOADED ==")
  return dataset

def saveDataframe(df,fileID):
  file_path = PATH+'GMF_filled_'+DATA[fileID]
  df.to_csv(file_path)

def genGMFmat(df,df_test,modelID,fileID,nbrE,lossF,OF,emb,filter=None,cb=None):

  # df_og = loadDataset(fileID)
  # df = processGmfData(df_og,'userID','itemID',1000,2000+df_og['reviewerID'].nunique(),60000,61000+df_og['productID'].nunique())
  
  x_train ,x_test,y_train,y_test = train_test_split(df_test[['userID','itemID']],df_test['rating'],test_size=0.2)
  model_trained = trainModel(modelID,nbrE,lossF,OF,[x_train['userID'],x_train['itemID']],y_train,maxUserID=df_test['userID'].max() + 1,maxItemID =df_test['itemID'].max() + 1,embed_size=emb)
  model_trained.evaluate([x_test['userID'],x_test['itemID']],y_test)

  #df_mat_filled = FillSparseMat(model_trained,df_test,df)
  
  return model_trained

def FillSparseMat(model,df_prepro,df_og):

  user_s,item_s = user_item_ID_lists(df_prepro['userID'].unique(),df_prepro['itemID'].unique())
  #df_mat_filled = Fill_Cf_Matrix(model,user_s,item_s,df_og['userID'].unique(),df_og['itemID'].unique())

  return user_s,item_s


def trainModel(modelID,nbrEpochs,lossF,OF,x_train ,y_train ,mid_layer_ratio=None,nb_layers=None,maxUserID = None,maxItemID = None,embed_size = None,filter_size = None,bs = 16):
  
  if modelID =='BLCNN':
    model = getBLCNNmodel(embed_size,filter_size)
    
  if modelID == 'GMF':
    model = getGMFmodel(maxUserID,maxItemID,embed_size)
    print(model.summary())
  elif modelID == 'S-AutoCF':
    model = getAutoCFmodel(x_train,mid_layer_ratio,nb_layers)
    print(model.summary())
  elif modelID == 'SS-AutoCF':
    model = getSS_HAEmodel(x_train[0],x_train[1],mid_layer_ratio,nb_layers)

  model.compile(optimizer = OF,
                    loss = lossF,
                    metrics= METRICS)
  model.fit(x_train,y_train,epochs = nbrEpochs,batch_size = bs,validation_split=0.14)
  
  return model

def getAutoCFmodel(x_train,mid_layer_ratio,nb_layers):
  #mid_layer_ratio [0 - 1]
  layer_ratio =  mid_layer_ratio + nb_layers*0.05

  encoder_input = layers.Input(shape=(x_train.shape[1]),name='user_item')
  flat = layers.Flatten()(encoder_input)

  if nb_layers != 1:
    hid_encoder = layers.Dense(layer_ratio*x_train.shape[1],activation="sigmoid")(flat)
    for i in range(nb_layers-1):
      layer_ratio -= 0.05
      hid_encoder = layers.Dense(layer_ratio*x_train.shape[1],activation="sigmoid")(hid_encoder)
    encoder_output = layers.Dense(mid_layer_ratio*x_train.shape[1],activation="sigmoid")(hid_encoder)
  else:
    encoder_output = layers.Dense(mid_layer_ratio*x_train.shape[1],activation="sigmoid")(flat)

  if nb_layers != 1:
    decoder_input = layers.Dense(layer_ratio*x_train.shape[1],activation="sigmoid")(encoder_output)
    for i in range(nb_layers-1):
      layer_ratio += 0.05
      decoder_input = layers.Dense(layer_ratio*x_train.shape[1],activation="sigmoid")(decoder_input)
    decoder_output = layers.Dense(x_train.shape[1],activation="sigmoid")(decoder_input)
  else:
    decoder_output = layers.Dense(x_train.shape[1],activation="sigmoid")(encoder_output)

  autoencoder = tf.keras.Model(inputs = encoder_input, outputs = decoder_output)

  return autoencoder

def getSS_HAEmodel(x_train_rat,x_train_trust,mid_layer_ratio,nb_layers):
  
  #mid_layer_ratio [0 - 1]
  layer_ratio =  mid_layer_ratio + nb_layers*0.1

  #Social_Autoencoder
  #layer_ratio*(x_train_rat.shape[1]+x_train_trust.shape[1]
  rating_input = layers.Input(shape=(x_train_rat.shape[1]),name='user_item')
  social_input = layers.Input(shape=(x_train_trust.shape[1]),name='user_user')

  flat_rating = layers.Flatten()(rating_input)
  flat_social = layers.Flatten()(social_input)

  SharedLayer_encoder = layers.Concatenate()([flat_rating,flat_social])

  if nb_layers !=1:
    hid_encoder = layers.Dense(layer_ratio*(x_train_rat.shape[1]+x_train_trust.shape[1]),activation="sigmoid")(SharedLayer_encoder)
    for i in range(nb_layers-1):
      layer_ratio -= 0.1
      hid_encoder = layers.Dense(layer_ratio*(x_train_rat.shape[1]+x_train_trust.shape[1]),activation="sigmoid")(hid_encoder)
    encoder_output = layers.Dense(mid_layer_ratio*(x_train_rat.shape[1]+x_train_trust.shape[1]),activation="sigmoid")(hid_encoder)
  else:
    encoder_output = layers.Dense(mid_layer_ratio*(x_train_rat.shape[1]+x_train_trust.shape[1]),activation="sigmoid")(SharedLayer_encoder)
  if nb_layers !=1:
    hid_decoder = layers.Dense(layer_ratio*(x_train_rat.shape[1]+x_train_trust.shape[1]),activation="sigmoid")(encoder_output)
    for i in range(nb_layers-1):
      layer_ratio += 0.1
      hid_decoder = layers.Dense(layer_ratio*(x_train_rat.shape[1]+x_train_trust.shape[1]),activation="sigmoid")(hid_decoder)
    SharedLayer_decoder =  layers.Dense(x_train_rat.shape[1]+x_train_trust.shape[1],activation="sigmoid")(hid_decoder)
  else:
    SharedLayer_decoder =  layers.Dense(x_train_rat.shape[1]+x_train_trust.shape[1],activation="sigmoid")(encoder_output)

  rating_decoded , social_decoded = tf.split(SharedLayer_decoder,[x_train_rat.shape[1],x_train_trust.shape[1]],1)

  # rating_output = layers.Dense(x_train_rat.shape[1],activation="relu",name='rating_output')(SharedLayer_decoder)
  # social_output = layers.Dense(x_train_trust.shape[1],activation="relu",name='social_output')(SharedLayer_decoder)

  autoencoder = tf.keras.Model(inputs = [rating_input,social_input], outputs = [rating_decoded,social_decoded])

  return autoencoder

def createAutoCF(modelID,input_fileID,target_fileID,nbrEpochs,lossF,OF,mid_layer_ratio,nb_layers,bs):

  sparseDf = loadDataset(input_fileID)  
  sparseDf.drop_duplicates(subset =['itemID', 'userID'] , keep = 'first' , inplace = True)
  df_mat_filled = loadDataset(target_fileID)
  df_mat_filled = df_mat_filled.set_index('Unnamed: 0')
  df_mat = DatasetToUserItemDataFrame(sparseDf,'userID','itemID','rating')
  # if df_mat.shape[0]< df_mat.shape[1]:
  #   df_mat = df_mat.T
  #   df_mat_filled = df_mat_filled.T
  #   print(df_mat_filled.shape,df_mat.shape)
  x_train,x_test,y_train,y_test = train_test_split(df_mat,df_mat_filled,test_size = 0.2)
  model_trained = trainModel(modelID,nbrEpochs,lossF,OF,x_train,y_train,mid_layer_ratio,nb_layers,bs = bs)

  return model_trained , x_test, y_test

def createSS_HAE(modelID,input_rating_fileID,target_rating_fileID,trust_fileID,nbrEpochs,lossF,OF,mid_layer_ratio,nb_layers,bs):

  sparseDf = loadDataset(input_rating_fileID)
  sparseDf.drop_duplicates(subset =['itemID', 'userID'] , keep = 'first' , inplace = True)
  df_rating_filled = loadDataset(target_rating_fileID)
  df_rating_filled = df_rating_filled.set_index('Unnamed: 0')
  df_trust_mat = loadDataset(trust_fileID)
  df_trust_mat = df_trust_mat.set_index('Unnamed: 0.1')
  df_trust_mat.drop(columns=['Unnamed: 0'],inplace=True)
  # trust_users_list = list(df_trust_mat.index)
  # df_rating_filled = getUsersRatings(df_rating_filled,trust_users_list) 
  # df_rating_filled = orgDataframe(df_rating_filled,trust_users_list)

  #input & target for autoencoder training
  df_rating_mat = DatasetToUserItemDataFrame(sparseDf,'userID','itemID','rating')
  # df_rating_mat_filled = orgMatDataframe(df_rating_mat_filled,trust_users_list)

  #data split
  x_rating_train,x_rating_test,y_rating_train,y_rating_test = train_test_split(df_rating_mat,df_rating_filled)
  x_train,x_test = train_test_split(df_trust_mat)

  model_trained = trainModel(modelID,nbrEpochs,lossF,OF,[x_rating_train,x_train],[y_rating_train,x_train],mid_layer_ratio,nb_layers,bs=bs)

  return model_trained , [x_rating_test,y_rating_test] , x_test

def evaluateModel(model,x_test,y_test):
  model.evaluate(x_test,y_test) 

def orgDataframe(df_to_org,org_list):

  df_org = pd.DataFrame(columns = ['userID','itemID','rating','reviewText'])
  for user in org_list:
    df_temp = df_to_org[(df_to_org['userID'] == user )]
    df_org = pd.concat([df_org,df_temp])

  return df_org

def orgMatDataframe(df_to_org,org_list):
  
  df_org = pd.DataFrame(index = list(df_to_org.index),columns = list(df_to_org.columns))
  for user in org_list:
    df_org.loc[user] = list(df_to_org.loc[user])
  
  return df_org

def getItemsScore(userID,modelID,fileID):
  
  listeItemScore = pd.DataFrame()
  
  if modelID == 0:
    listItemScore = predSeCF(userID,modelID,fileID)   #CF
  elif modelID == 1:
    listItemScore = predSeCF(userID,modelID,fileID)   #CF + Sentiment
  elif modelID == 2:
    listItemScore = predSSeCF(userID,modelID,fileID)  #CF + Social
  elif modelID == 3:
    listItemScore = predSSeCF(userID,modelID,fileID)  #CF + Sentiment + Social

  return listeItemScore 

def getModel(modelID):
  model = tf.keras.models.load_model(MODEL_PATH[modelID])
  return model

def loadDataFrame(df):
  df_loaded = pd.read_csv(DATA[df])
  return  df_loaded

def predSeCF(userID,modelID,fileID):
  
  model = getModel(modelID)
  df_user_item_mat = pd.read_csv(DATA[fileID])
  SparseScoresVec = df_user_item_mat.loc[userID].to_numpy()
  listItemScores = df_user_item_mat.loc[userID].to_frame()
  listItemScores = listItemScores.reset_index()
  listItemScores.set_axis(['itemID','score'],axis='columns',inplace=True)
  PredScoresVec = model.predict(SparseScoresVec)
  listItemScores.replace(SparseScoresVec,PredScoresVec,inplace = True)

  return listItemScores

def predSSeCF(userID,modelID,ratings_fileID,trust_fileID):

  model = getModel(modelID)
  df_user_item_mat = loadDataset(fileID)
  df_user_user_mat = pd.read_csv(trust_fileID)
  SparseScoresVec = df_user_item_mat.loc[userID].to_numpy()
  SparseTrustVec = df_user_user_mat.loc[userID].to_numpy()
  listItemScores = df_user_item_mat.loc[userID].to_frame()
  listItemScores = listItemScores.reset_index()
  listItemScores.set_axis(['itemID','score'],axis='columns',inplace=True)
  PredScoresVec,SparseTrustVec = model.predict([SparseScoresVec,SparseTrustVec])
  listItemScores.replace(SparseScoresVec,PredScoresVec,inplace = True)

  return listItemScores
  

In [ ]:
def datapreprocess(df_sample):
  
  print("== PREPROCESSING DATA ...")
  df_sample = generateIDs(df_sample,'userID','itemID',5000,6000+df_sample['userID'].nunique(),8000,8000+df_sample['itemID'].nunique())
  
  df_sample['one'] = df_sample['rating'].apply(lambda x: 1 if x==1 else 0)
  df_sample['two'] = df_sample['rating'].apply(lambda x: 1 if x==2 else 0)
  df_sample['three'] = df_sample['rating'].apply(lambda x: 1 if x==3 else 0)
  df_sample['four'] = df_sample['rating'].apply(lambda x: 1 if x==4 else 0)
  df_sample['five'] = df_sample['rating'].apply(lambda x: 1 if x==5 else 0)
  #df['six'] = df['rating'].apply(lambda x: 1 if x==6 else 0)
  print("== DATA PREPROCESSED ==")

  return df_sample

In [18]:
def normalize(df):
  transformer = ColumnTransformer(transformers=[('norm', preprocessing.MinMaxScaler(feature_range=(0, 1)), ['rating'])])
  col = transformer.fit_transform(df)
  return col

In [ ]:
import tensorflow_ranking as tfr

In [29]:
DATA = ['/content/review_restaurant_norm.csv','/content/Yelp_restaurants_GMF_filled_mat.csv','/content/gdrive/MyDrive/RESTAURANT_Final_UserUser_trust_DF.csv','/content/Yelp_shopping_GMF_filled_mat.csv']
MODEL_PATH = []
PATH = ''
rmse = tf.keras.metrics.RootMeanSquaredError()
precision = tf.keras.metrics.Precision()
# ndcg = tfr.keras.metrics.NDCGMetric()
METRICS = ['mae',rmse]

In [25]:
df_ = pd.read_csv('/content/review_restaurant_norm.csv')

In [ ]:
df = yelpPreprocess(df)

In [19]:
col = normalize(df)

In [23]:
df.to_csv('/content/review_restaurant_norm.csv',index=False)

In [ ]:
model,x_test,y_test = createAutoCF('S-AutoCF',0,1,10,tf.keras.losses.MeanSquaredError(),'adam',0.3,1,8)

In [ ]:
evaluateModel(model,x_test,y_test)

In [ ]:
model,[x_rating_test,y_rating_test],y_test = createSS_HAE('SS-AutoCF',0,1,2,10,tf.keras.losses.MeanSquaredError(),'adam',0.3,1,8)

In [ ]:
evaluateModel(model,[x_rating_test,y_test],[y_rating_test,y_test])

34/34 [==============================] - 1s 22ms/step - loss: 0.0145 - tf.split_loss: 0.0129 - tf.split_1_loss: 0.0016 - tf.split_mae: 0.0872 - tf.split_root_mean_squared_error: 0.1137 - tf.split_1_mae: 0.0199 - tf.split_1_root_mean_squared_error: 0.0395


In [ ]:
model.save("/content/gdrive/MyDrive/CF")

In [ ]:
predicted_test = model.predict(x_test)
y_test_array = np.array(y_test)
# precision_10 = tf.keras.metrics.Precision(top_k =10)
ndcg10 = tfr.keras.metrics.NDCGMetric(topn=10)
ndcg10.update_state(y_test_array,predicted_test)
# precision_10.update_state(y_test_array,predicted_test)
ndcg10.result().numpy()

0.45489812

In [ ]:
df_sample = pd.read_csv("/content/review_shopping.csv.zip")

In [ ]:
df_sample.rename(columns = {'text': 'reviewText', 'stars': 'rating', 'business_id': 'itemID', 'user_id': 'userID'}, inplace = True)
df_sample.dropna(how='any',axis=0)
df_sample.drop_duplicates(subset =['itemID', 'userID'] , keep = 'first' , inplace = True)

In [ ]:
df_sample.head()

In [ ]:
df_test = datapreprocess(df_sample)

== PREPROCESSING DATA ...


100% (4346 of 4346) |####################| Elapsed Time: 0:02:07 Time:  0:02:07
100% (15588 of 15588) |##################| Elapsed Time: 0:14:52 Time:  0:14:52


== DATA PREPROCESSED ==


In [ ]:
df_test.head()

In [ ]:
model = genGMFmat(df_sample,df_test,'GMF',0,2,tf.keras.losses.MeanSquaredError(),'adam',32)

In [ ]:
user_s,item_s = FillSparseMat(model,df_test,df_sample)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:24: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:25: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
100% (4346 of 4346) |####################| Elapsed Time: 0:27:32 Time:  0:27:32


In [ ]:
df_mat_filled = Fill_Cf_Matrix(model,user_s,item_s,df_sample['userID'].unique(),df_sample['itemID'].unique())

2117046/2117046 [==============================] - 2817s 1ms/step


  0% (58217 of 67745448) |               | Elapsed Time: 0:00:00 ETA:   0:03:52

pred done


100% (67745448 of 67745448) |############| Elapsed Time: 0:03:48 Time:  0:03:48


In [ ]:
df_mat_filled.to_csv("/content/Yelp_shopping_GMF_filled_mat.csv")

In [ ]:
from google.colab import files
files.download("/content/Yelp_shopping_GMF_filled_mat.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>